<a href="https://colab.research.google.com/github/sreent/machine-learning/blob/main/Final%20DNN%20Code%20Examples/Fashion%20MNIST/Fashion%20MNIST%20-%20TFDS%20Gray-Scaled%20Image%20Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fashion MNIST - TFDS Grayscale Image Example

This notebook demonstrates the **Universal ML Workflow** applied to **multi-class image classification** using grayscale images from TensorFlow Datasets.

## Learning Objectives

By the end of this notebook, you will be able to:
- Work with **grayscale images** (no colour conversion needed)
- Handle **10-class image classification**
- Understand **image preprocessing** for dense neural networks (resize + flatten)
- Use **TensorFlow Datasets** for standard benchmarks
- Use **Hyperband** for efficient hyperparameter tuning
- Apply **Dropout + L2 regularisation** to prevent overfitting

---

## Dataset Overview

| Attribute | Description |
|-----------|-------------|
| **Source** | [TensorFlow Datasets - fashion_mnist](https://www.tensorflow.org/datasets/catalog/fashion_mnist) |
| **Problem Type** | Multi-Class Classification (10 classes) |
| **Classes** | T-shirt, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot |
| **Data Balance** | Perfectly Balanced (7,000 per class) |
| **Image Size** | 28×28 grayscale (resized to 16×16) |
| **Preprocessing** | Resize → Flatten → Normalise |

---

## Technique Scope

This notebook uses only techniques from **Chapters 1–4** of *Deep Learning with Python* (Chollet, 2021). This means:

| Technique | Status | Rationale |
|-----------|--------|-----------|
| **Dense layers (DNN)** | ✓ Used | Core building block (Ch. 3-4) |
| **Dropout** | ✓ Used | Regularisation technique (Ch. 4) |
| **L2 regularisation** | ✓ Used | Weight penalty (Ch. 4) |
| **Early stopping** | ✗ Not used | Introduced in Ch. 7 |
| **CNN** | ✗ Not used | Introduced in Ch. 8 |
| **RNN/LSTM** | ✗ Not used | Introduced in Ch. 10 |

**Note:** For image classification, Convolutional Neural Networks (CNNs) would typically be preferred. Here we use **Dense layers only** to demonstrate the Universal ML Workflow and regularisation techniques. Images are flattened to 1D vectors before being fed to the network.

---

## Why Fashion MNIST?

Fashion MNIST is a drop-in replacement for classic MNIST digit classification:

| Aspect | MNIST (Digits) | Fashion MNIST |
|--------|----------------|---------------|
| **Classes** | 10 digits (0-9) | 10 clothing items |
| **Difficulty** | Essentially "solved" (~99.8%) | More challenging (~92-94% with CNN) |
| **Realism** | Handwritten digits | Real product images |
| **Image size** | 28×28 grayscale | 28×28 grayscale |

---

## 1. Defining the Problem and Assembling a Dataset

The first step in any machine learning project is to clearly define the problem and understand the data.

**Problem Statement:** Classify grayscale images of fashion items into 10 categories.

**Why this problem is interesting:**
- **Standard benchmark:** Fashion MNIST is widely used for evaluating classification algorithms
- **Balanced classes:** Enables straightforward accuracy comparison
- **Moderate difficulty:** More challenging than digit MNIST, but achievable with basic techniques

**Data Source:** TensorFlow Datasets provides pre-split train/test sets directly.

## 2. Choosing a Measure of Success

### Metric Selection Based on Class Imbalance

| Imbalance Ratio | Classification | Primary Metric | Rationale |
|-----------------|----------------|----------------|-----------|
| ≤ 1.5:1 | Balanced | **Accuracy** | Classes roughly equal |
| 1.5:1 – 3:1 | Mild Imbalance | **Accuracy** | Majority class < 75% |
| > 3:1 | Moderate/Severe | **F1-Score** | Accuracy becomes misleading |

**Fashion MNIST is perfectly balanced** (7,000 samples per class), so **Accuracy** is our primary metric.

We also report **Top-K Accuracy** - was the correct class among the top K predictions? This is useful for multi-class problems where similar items (e.g., Pullover vs. Coat) may be easily confused.

## 3. Deciding on an Evaluation Protocol

### Hold-Out vs K-Fold Cross-Validation

| Dataset Size | Recommended Method | Rationale |
|--------------|-------------------|-----------|
| < 1,000 | K-Fold (K=5 or 10) | High variance with small hold-out sets |
| 1,000 – 10,000 | K-Fold or Hold-Out | Either works; K-fold more robust |
| > 10,000 | Hold-Out | Sufficient data; K-fold computationally expensive |

### Data Split Strategy (This Notebook)

```
Original Data (70,000 samples) → Hold-Out Selected
├── Test Set (10%) - Final evaluation only
└── Training Pool (90%)
    ├── Training Set (~81%) - Model training
    └── Validation Set (~9%) - Hyperparameter tuning
```

**Important:** We use `stratify` parameter to maintain class proportions in all splits.

### References

- Chollet, F. (2021) *Deep learning with Python*. 2nd edn. Shelter Island, NY: Manning Publications.

- Kohavi, R. (1995) 'A study of cross-validation and bootstrap for accuracy estimation and model selection', *IJCAI*, 2, pp. 1137–1145.

## 4. Preparing Your Data

### 4.1 Import Libraries and Set Random Seed

We set random seeds for reproducibility - this ensures that running the notebook multiple times produces the same results.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds

# Keras Tuner for hyperparameter search
%pip install -q -U keras-tuner
import keras_tuner as kt

import matplotlib.pyplot as plt

SEED = 204

tf.random.set_seed(SEED)
np.random.seed(SEED)

import warnings
warnings.filterwarnings('ignore')

### 4.2 Load Dataset from TensorFlow Datasets

TensorFlow Datasets provides convenient access to standard ML benchmarks.

In [ ]:
# Dataset configuration
DATASET = 'fashion_mnist'
RESIZE = (16, 16)  # Resize 28x28 to 16x16 to reduce features

# Class names for Fashion MNIST
CLASS_NAMES = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f"Loading {DATASET} from TensorFlow Datasets...")

In [ ]:
# Load dataset
ds = tfds.load(DATASET, split='all', shuffle_files=True)

# Extract images and labels
images, labels = [], []
for entry in ds.take(len(ds)):
    image, label = entry['image'], entry['label']
    
    # Convert to numpy (grayscale has 1 channel)
    image = image.numpy()[:, :, 0]
    label = label.numpy()
    
    # Resize image to reduce dimensions
    image = resize(image, RESIZE, anti_aliasing=True)
    
    images.append(image)
    labels.append(label)

print(f"Loaded {len(images):,} images")
print(f"Original size: 28×28, Resized to: {RESIZE[0]}×{RESIZE[1]}")

In [ ]:
# Display sample images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title(CLASS_NAMES[labels[i]])
    ax.axis('off')
plt.suptitle('Sample Fashion MNIST Images (resized to 16×16)', fontsize=12)
plt.tight_layout()
plt.show()

### 4.3 Prepare Features and Labels

**Image Preprocessing for Dense Networks:**

```
28×28 Image → Resize to 16×16 → Flatten to 256 features → Normalise [0, 1]
```

**Why flatten?** Dense layers expect 1D input. Without CNNs (which preserve spatial structure), we treat each pixel as an independent feature.

**Why resize?** Reducing from 784 (28×28) to 256 (16×16) features:
- Reduces model parameters and training time
- Helps prevent overfitting on small patterns
- 16×16 retains enough detail for classification

In [ ]:
# Convert to numpy arrays and flatten
X = np.array(images)
X = X.reshape((X.shape[0], -1))  # Flatten: (N, 16, 16) -> (N, 256)

print(f"Feature shape: {X.shape}")
print(f"Features per image: {X.shape[1]} ({RESIZE[0]}×{RESIZE[1]} = {RESIZE[0]*RESIZE[1]})")

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
label_encoder.fit(labels)

# One-hot encode for multi-class classification
y = to_categorical(label_encoder.transform(labels))

print(f"Label shape: {y.shape}")
print(f"Number of classes: {y.shape[1]}")

### 4.4 Split Data into Train and Test Sets

In [ ]:
TEST_SIZE = 0.10

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, 
    stratify=labels,  # Maintain class balance
    shuffle=True, random_state=SEED
)

print(f"Training pool: {len(X_train_full):,} samples")
print(f"Test set: {len(X_test):,} samples")

In [ ]:
# Normalise pixel values from [0, 1] (after resize) to ensure consistent range
X_train_full = X_train_full / X_train_full.max()
X_test = X_test / X_test.max()

print(f"Pixel value range: [{X_train_full.min():.2f}, {X_train_full.max():.2f}]")

## 5. Developing a Model That Does Better Than a Baseline

Before building complex models, we need to establish **baseline performance**.

### 5.1 Examine Class Distribution

In [ ]:
# Count samples per class
counts = np.sum(y, axis=0)
print("Samples per class:")
for i, (name, count) in enumerate(zip(CLASS_NAMES, counts)):
    print(f"  {i}: {name}: {int(count):,}")

In [ ]:
# =============================================================================
# DATA-DRIVEN ANALYSIS: Dataset Size & Imbalance
# =============================================================================

# Dataset size analysis
n_samples = len(X)
n_classes = y.shape[1]
HOLDOUT_THRESHOLD = 10000

# Imbalance analysis
imbalance_ratio = counts.max() / counts.min()
IMBALANCE_THRESHOLD = 3.0

use_holdout = n_samples > HOLDOUT_THRESHOLD
use_accuracy = imbalance_ratio <= IMBALANCE_THRESHOLD

print("=" * 60)
print("DATA-DRIVEN CONFIGURATION")
print("=" * 60)
print(f"\n1. DATASET SIZE: {n_samples:,} samples")
print(f"   Threshold: {HOLDOUT_THRESHOLD:,} samples")
print(f"   Decision: {'Hold-Out' if use_holdout else 'K-Fold Cross-Validation'}")

print(f"\n2. CLASS IMBALANCE: {imbalance_ratio:.2f}:1 ratio")
print(f"   Threshold: {IMBALANCE_THRESHOLD:.1f}:1")
print(f"   Decision: {'Accuracy (balanced)' if use_accuracy else 'F1-Score (imbalanced)'}")

print(f"\n3. NUMBER OF CLASSES: {n_classes}")

print("\n" + "=" * 60)
PRIMARY_METRIC = 'accuracy' if use_accuracy else 'f1'
print(f"PRIMARY METRIC: {PRIMARY_METRIC.upper()}")
print("=" * 60)

In [ ]:
# Baseline accuracy (random guessing with balanced classes)
baseline = 1.0 / n_classes

print(f"Baseline accuracy (random guess): {baseline:.2%}")

### 5.2 Create Validation Set

In [ ]:
VALIDATION_SIZE = 0.10  # 10% of training pool

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, 
    test_size=VALIDATION_SIZE, 
    stratify=y_train_full.argmax(axis=1),
    shuffle=True, random_state=SEED
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Validation set: {X_val.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")

### 5.3 Configure Training Parameters

In [ ]:
INPUT_DIMENSION = X_train.shape[1]
OUTPUT_CLASSES = y_train.shape[1]

OPTIMIZER = 'adam'
LOSS_FUNC = 'categorical_crossentropy'  # Multi-class classification

# Training metrics
METRICS = ['accuracy']

print(f"Input dimension: {INPUT_DIMENSION}")
print(f"Output classes: {OUTPUT_CLASSES}")

In [ ]:
# Single-Layer Perceptron (no hidden layers) - Baseline
slp_model = Sequential(name='Single_Layer_Perceptron')
slp_model.add(layers.Input(shape=(INPUT_DIMENSION,)))
slp_model.add(Dense(OUTPUT_CLASSES, activation='softmax'))
slp_model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNC, metrics=METRICS)

slp_model.summary()

In [ ]:
# =============================================================================
# TRAINING CONFIGURATION
# =============================================================================

BATCH_SIZE = 512

# Epoch strategy:
# EPOCHS_BASELINE (100): For SLP and unregularised DNN
# EPOCHS_REGULARIZED (150): For DNN with Dropout + L2

EPOCHS_BASELINE = 100
EPOCHS_REGULARIZED = 150

In [ ]:
# Train the Single-Layer Perceptron
history_slp = slp_model.fit(
    X_train, y_train, 
    batch_size=BATCH_SIZE, epochs=EPOCHS_BASELINE, 
    validation_data=(X_val, y_val),
    verbose=0
)
val_score_slp = slp_model.evaluate(X_val, y_val, verbose=0)

In [ ]:
# Display SLP validation metrics
preds_slp_val = slp_model.predict(X_val, verbose=0).argmax(axis=1)
acc_slp_val = accuracy_score(y_val.argmax(axis=1), preds_slp_val)

print('Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(acc_slp_val, baseline))
print(f'\nAccuracy: {acc_slp_val:.2%}  ← Primary Metric')

In [ ]:
def plot_training_history(history, title=None):
    """
    Plot training and validation metrics over epochs.
    Plots: (1) Loss, (2) Accuracy
    """
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))
    epochs = range(1, len(history.history['loss']) + 1)
    title_suffix = f' ({title})' if title else ''

    # Plot 1: Loss
    axs[0].plot(epochs, history.history['loss'], 'b-', label='Training', linewidth=1.5)
    axs[0].plot(epochs, history.history['val_loss'], 'r-', label='Validation', linewidth=1.5)
    axs[0].set_title(f'Loss{title_suffix}')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    axs[0].grid(alpha=0.3)

    # Plot 2: Accuracy
    axs[1].plot(epochs, history.history['accuracy'], 'b-', label='Training', linewidth=1.5)
    axs[1].plot(epochs, history.history['val_accuracy'], 'r-', label='Validation', linewidth=1.5)
    axs[1].set_title(f'Accuracy{title_suffix}')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()
    axs[1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot SLP training history
plot_training_history(history_slp, title='SLP Baseline')

## 6. Scaling Up: Developing a Model That Overfits

The next step is to build a model with **enough capacity to overfit**.

**No regularisation applied:** We train without regularisation to observe overfitting.

### 6.1 Build a Deep Neural Network (DNN)

In [ ]:
# Deep Neural Network (1 hidden layer, no regularisation)
dnn_model = Sequential(name='Deep_Neural_Network')
dnn_model.add(layers.Input(shape=(INPUT_DIMENSION,)))
dnn_model.add(Dense(64, activation='relu'))
dnn_model.add(Dense(OUTPUT_CLASSES, activation='softmax'))
dnn_model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNC, metrics=METRICS)

dnn_model.summary()

In [ ]:
# Train the Deep Neural Network
history_dnn = dnn_model.fit(
    X_train, y_train, 
    batch_size=BATCH_SIZE, epochs=EPOCHS_BASELINE, 
    validation_data=(X_val, y_val), 
    verbose=0
)
val_score_dnn = dnn_model.evaluate(X_val, y_val, verbose=0)

In [ ]:
# Plot DNN training history
plot_training_history(history_dnn, title='DNN - No Regularisation')

In [ ]:
# Display DNN validation metrics
preds_dnn_val = dnn_model.predict(X_val, verbose=0).argmax(axis=1)
acc_dnn_val = accuracy_score(y_val.argmax(axis=1), preds_dnn_val)

print('Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(acc_dnn_val, baseline))
print(f'\nAccuracy: {acc_dnn_val:.2%}  ← Primary Metric')

## 7. Regularising Your Model and Tuning Hyperparameters

Now we address overfitting by adding **Dropout + L2 regularisation**.

Using **Hyperband** for efficient hyperparameter tuning.

### 7.1 Hyperband Search

In [ ]:
# Hyperband Model Builder
def build_model_hyperband(hp):
    """
    Build Fashion MNIST model with FIXED architecture (1 hidden layer, 64 neurons).
    Only tunes regularisation and learning rate.
    """
    model = keras.Sequential()
    model.add(layers.Input(shape=(INPUT_DIMENSION,)))

    # L2 regularisation strength
    l2_reg = hp.Float('l2_reg', 1e-5, 1e-2, sampling='log')

    # Fixed architecture: 1 hidden layer with 64 neurons
    model.add(layers.Dense(64, activation='relu', 
                           kernel_regularizer=regularizers.l2(l2_reg)))
    dropout_rate = hp.Float('dropout', 0.0, 0.5, step=0.1)
    model.add(layers.Dropout(dropout_rate))

    # Output layer for multi-class classification
    model.add(layers.Dense(OUTPUT_CLASSES, activation='softmax'))

    lr = hp.Float('lr', 1e-4, 1e-2, sampling='log')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=LOSS_FUNC,
        metrics=METRICS
    )
    return model

In [ ]:
# Configure Hyperband tuner
tuner = kt.Hyperband(
    build_model_hyperband,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='fashion_mnist_hyperband',
    project_name='fashion_mnist_tuning',
    overwrite=True
)

print("Tuning objective: val_accuracy")

# Run Hyperband search
tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=BATCH_SIZE
)

In [ ]:
# Get best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
print(f"  L2 Regularisation: {best_hp.get('l2_reg'):.6f}")
print(f"  Dropout Rate: {best_hp.get('dropout')}")
print(f"  Learning Rate: {best_hp.get('lr'):.6f}")

opt_model = tuner.hypermodel.build(best_hp)
opt_model.summary()

### 7.2 Retrain with Optimised Hyperparameters

In [ ]:
# Train with extended epochs
history_opt = opt_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS_REGULARIZED,
    batch_size=BATCH_SIZE,
    verbose=0
)
val_score_opt = opt_model.evaluate(X_val, y_val, verbose=0)

In [ ]:
# Plot optimised model training history
plot_training_history(history_opt, title='DNN - Dropout + L2')

In [ ]:
# Display optimised model validation metrics
preds_opt_val = opt_model.predict(X_val, verbose=0).argmax(axis=1)
acc_opt_val = accuracy_score(y_val.argmax(axis=1), preds_opt_val)

print('Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(acc_opt_val, baseline))
print(f'\nAccuracy: {acc_opt_val:.2%}  ← Primary Metric')

### 7.3 Final Model Evaluation on Test Set

In [ ]:
# Final evaluation on test set
preds_test = opt_model.predict(X_test, verbose=0)
preds_test_labels = preds_test.argmax(axis=1)
y_test_labels = y_test.argmax(axis=1)

test_accuracy = accuracy_score(y_test_labels, preds_test_labels)
test_precision = precision_score(y_test_labels, preds_test_labels, average='macro')
test_recall = recall_score(y_test_labels, preds_test_labels, average='macro')
test_f1 = f1_score(y_test_labels, preds_test_labels, average='macro')

print('=' * 50)
print('FINAL TEST SET RESULTS')
print('=' * 50)
print(f'Accuracy (Test): {test_accuracy:.4f}  ← Primary Metric')
print(f'Precision (Test, macro): {test_precision:.4f}')
print(f'Recall (Test, macro): {test_recall:.4f}')
print(f'F1-Score (Test, macro): {test_f1:.4f}')
print(f'\nBaseline: {baseline:.4f}')

In [ ]:
# Confusion matrix
fig, ax = plt.subplots(figsize=(10, 8))
cm = confusion_matrix(y_test_labels, preds_test_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASS_NAMES)
disp.plot(ax=ax, cmap='Blues', values_format='d', xticks_rotation=45)
plt.title('Confusion Matrix - Test Set')
plt.tight_layout()
plt.show()

In [ ]:
# Top-K Accuracy
def top_k_accuracy(y_true, y_pred_proba, k):
    """Calculate top-K accuracy: correct class in top K predictions."""
    top_k_preds = np.argsort(y_pred_proba, axis=1)[:, -k:]
    correct = sum(y_true[i] in top_k_preds[i] for i in range(len(y_true)))
    return correct / len(y_true)

print("Top-K Accuracy (Test Set):")
for k in [1, 2, 3]:
    top_k = top_k_accuracy(y_test_labels, preds_test, k)
    print(f"  Top-{k}: {top_k:.2%}")

---

## 8. Results Summary

In [ ]:
# Results summary
results = pd.DataFrame({
    'Model': ['Random Baseline', 'SLP (No Hidden)', 'DNN (No Regularisation)', 'DNN (Dropout + L2)', 'DNN (Dropout + L2) - Test'],
    'Accuracy': [baseline, acc_slp_val, acc_dnn_val, acc_opt_val, test_accuracy],
    'Dataset': ['N/A', 'Validation', 'Validation', 'Validation', 'Test']
})

print("=" * 70)
print("MODEL COMPARISON - RESULTS SUMMARY")
print("=" * 70)
print(f"Primary Metric: ACCURACY (balanced dataset, {n_classes} classes)")
print("=" * 70)
print(results.to_string(index=False, float_format='{:.4f}'.format))
print("=" * 70)
print(f"\nKey Observations:")
print(f"  - All models significantly outperform random baseline ({baseline:.2%})")
print(f"  - DNN with regularisation achieves {test_accuracy:.2%} test accuracy")
print(f"  - Top-3 accuracy: {top_k_accuracy(y_test_labels, preds_test, 3):.2%}")

---

## 9. Key Takeaways

### Decision Framework Summary

| Decision | Threshold | This Dataset | Choice | Reference |
|----------|-----------|--------------|--------|-----------|
| **Hold-Out vs K-Fold** | > 10,000 samples | 70,000 samples | Hold-Out | Kohavi (1995) |
| **Accuracy vs F1-Score** | > 3:1 imbalance | 1.00:1 ratio | Accuracy | He and Garcia (2009) |

### Lessons Learned

1. **Image Preprocessing for DNNs:** Without CNNs, images must be flattened to 1D vectors. Resizing reduces dimensionality.

2. **Grayscale Simplifies Processing:** No colour conversion needed, single channel per pixel.

3. **Balanced Classes → Accuracy:** With 10 equal-sized classes, accuracy is a valid primary metric.

4. **Top-K Accuracy:** Useful for multi-class problems where similar items may be confused.

5. **Regularisation Prevents Overfitting:** Dropout + L2 controls overfitting without early stopping.

6. **Technique Scope:** Dense layers only (Ch. 1-4). CNNs would perform better but are introduced in Ch. 8.

### References

- Chollet, F. (2021) *Deep learning with Python*. 2nd edn. Shelter Island, NY: Manning Publications.

- He, H. and Garcia, E.A. (2009) 'Learning from imbalanced data', *IEEE Transactions on Knowledge and Data Engineering*, 21(9), pp. 1263–1284.

- Kohavi, R. (1995) 'A study of cross-validation and bootstrap for accuracy estimation and model selection', *IJCAI*, 2, pp. 1137–1145.

- Xiao, H., Rasul, K. and Vollgraf, R. (2017) 'Fashion-MNIST: a novel image dataset for benchmarking machine learning algorithms', *arXiv preprint arXiv:1708.07747*.